## Imports and Setup

In [1]:
import os
import sys
import joblib
import subprocess
sys.path.insert(0, "../utils/")
import pred_utils
import config
import geoutils
import eval_utils
sys.path.insert(0, "../")
import fusion

import pandas as pd
import numpy as np
import geopandas as gpd
from tqdm.notebook import tqdm

pd.set_option('mode.chained_assignment', None)

%load_ext autoreload
%autoreload 2

## Load Dataset

In [6]:
name = 'Saint Peter'
bldgs_file = '../data/vectors/building_footprints_annotated_DOM.gpkg'
bldgs = gpd.read_file(bldgs_file)

bldgs = bldgs[bldgs.shape_name == name]
print(f"Data dimensions: {bldgs.shape}")
print(bldgs.roof_type.value_counts())
bldgs.head(3)

Data dimensions: (1272, 8)
roof_type
GABLE       55
FLAT        12
HIP          9
NO_ROOF      6
HALF_HIP     1
Name: count, dtype: int64


,UID,Elevation,Layer,roof_type,roof_material,roof_condition,shape_name,geometry
34411,34413,17.535746,BLD_GEN_OUTLINE,NaN,NaN,NaN,Saint Peter,MULTIPOLYGON Z (((665583.468 1710787.345 14.89...
34476,34478,5.934477,BLD_GEN_OUTLINE,NaN,NaN,NaN,Saint Peter,MULTIPOLYGON Z (((665565.273 1710859.963 5.669...
34487,34489,6.699960,BLD_RESIDENTIAL_TRAILER,NaN,NaN,NaN,Saint Peter,MULTIPOLYGON Z (((665534.946 1710892.122 6.222...


## Generate Post-disaster Predictions

In [8]:
in_file = '../data/rasters/Ortho/ortho_DOM.tif'
exp_config = '../configs/config_09.yaml'
c = config.create_config(exp_config)
exp_dir = f"../exp/{c['exp_name']}/"
classes = geoutils.classes_dict[c['attribute']]
print(f"Config: {c}")

model = pred_utils.load_model(c, exp_dir, n_classes=len(classes))
#bldgs = pred_utils.generate_predictions(bldgs, model, c, in_file, exp_dir, classes=classes)   
#bldgs.head(3)
?model

Config: {'data_dir': './data/rasters/Tiles/ortho/', 'csv_dir': './data/csv/', 'attribute': 'roof_material', 'exp_name': 'exp-09', 'mode': 'RGB', 'batch_size': 32, 'n_workers': 4, 'n_epochs': 30, 'model': 'efficientnetb0', 'pretrained': True, 'scheduler': 'ReduceLROnPlateau', 'optimizer': 'Adam', 'lr': 1e-05, 'img_size': 224, 'momentum': None, 'gamma': None, 'step_size': None, 'patience': 7, 'dropout': None}
Model file ../exp/exp-09/best_model.pth successfully loaded.


Signature:       model(*args, **kwargs)
Type:            EfficientNet
String form:    
EfficientNet(
           (features): Sequential(
           (0): Conv2dNormActivation(
           (0): Conv2d(3, 32, k <...> : Dropout(p=0.2, inplace=True)
           (1): Linear(in_features=1280, out_features=5, bias=True)
           )
           )
File:            c:\users\issa\miniconda3\envs\gfdrr\lib\site-packages\torchvision\models\efficientnet.py
Docstring:       <no docstring>
Class docstring:
Base class for all neural network modules.

Your models should also subclass this class.

Modules can also contain other Modules, allowing to nest them in
a tree structure. You can assign the submodules as regular attributes::

    import torch.nn as nn
    import torch.nn.functional as F

    class Model(nn.Module):
        def __init__(self):
            super().__init__()
            self.conv1 = nn.Conv2d(1, 20, 5)
            self.conv2 = nn.Conv2d(20, 20, 5)

        def forward(self, x):
        

In [7]:
in_file = '../data/rasters/DSM/ndsm_DOM.tif'
exp_config = '../configs/config_05.yaml'
c = config.create_config(exp_config)
exp_dir = f"../exp/{c['exp_name']}/"
classes = geoutils.classes_dict[c['attribute']]
print(f"Config: {c}")

model = pred_utils.load_model(c, exp_dir, n_classes=len(classes))
bldgs = pred_utils.generate_predictions(bldgs, model, c, in_file, exp_dir, classes=classes)   
bldgs.head(3)

Config: {'data_dir': './data/rasters/Tiles/ndsm/', 'csv_dir': './data/csv/', 'attribute': 'roof_type', 'mode': 'GRAYSCALE', 'exp_name': 'exp-05', 'batch_size': 32, 'n_workers': 4, 'n_epochs': 30, 'model': 'resnet50', 'pretrained': True, 'scheduler': 'ReduceLROnPlateau', 'optimizer': 'Adam', 'lr': 1e-05, 'img_size': 224, 'momentum': None, 'gamma': None, 'step_size': None, 'patience': 7, 'dropout': 0.5}
Model file ../exp/exp-05/best_model.pth successfully loaded.


100%|███████████████| 1272/1272 [09:37<00:00,  2.20it/s]                                                               


,UID,Elevation,Layer,roof_type,roof_material,roof_condition,shape_name,geometry
34411,34413,17.535746,BLD_GEN_OUTLINE,HIP,HEALTHY_METAL,NaN,Saint Peter,MULTIPOLYGON Z (((665583.468 1710787.345 14.89...
34476,34478,5.934477,BLD_GEN_OUTLINE,GABLE,HEALTHY_METAL,NaN,Saint Peter,MULTIPOLYGON Z (((665565.273 1710859.963 5.669...
34487,34489,6.699960,BLD_RESIDENTIAL_TRAILER,FLAT,HEALTHY_METAL,NaN,Saint Peter,MULTIPOLYGON Z (((665534.946 1710892.122 6.222...


In [4]:
out_path = '../output/'
if not os.path.isdir(out_path):
    os.makedirs(out_path)
out_file = os.path.join(out_path, f'{name}-ortho-20230612.gpkg')
bldgs.to_file(out_file, driver='GPKG')

## Generate Pre-disaster Predictions

In [3]:
in_file = '../data/rasters/Drone/drone_colihaut_DOM.tif'
exp_config = '../configs/config_09.yaml'
c = config.create_config(exp_config)
exp_dir = f"../exp/{c['exp_name']}/"
classes = geoutils.classes_dict[c['attribute']]
print(f"Config: {c}")

model = pred_utils.load_model(c, exp_dir, n_classes=len(classes))
bldgs = pred_utils.generate_predictions(bldgs, model, c, in_file, exp_dir, classes=classes)   
bldgs.head(3)

Config: {'data_dir': './data/rasters/Tiles/ortho/', 'csv_dir': './data/csv/', 'attribute': 'roof_material', 'exp_name': 'exp-09', 'mode': 'RGB', 'batch_size': 32, 'n_workers': 4, 'n_epochs': 30, 'model': 'efficientnetb0', 'pretrained': True, 'scheduler': 'ReduceLROnPlateau', 'optimizer': 'Adam', 'lr': 1e-05, 'img_size': 224, 'momentum': None, 'gamma': None, 'step_size': None, 'patience': 7, 'dropout': None}
Model file ../exp/exp-09/best_model.pth successfully loaded.


100%|███████████████| 373/373 [04:09<00:00,  1.49it/s]                                                                 


,UID,roof_type,roof_material,geometry
4,34876,NO_ROOF,CONCRETE_CEMENT,"POLYGON Z ((665060.888 1712479.780 35.786, 665..."
5,34887,GABLE,HEALTHY_METAL,"POLYGON Z ((665078.259 1712486.585 32.366, 665..."
6,34890,GABLE,HEALTHY_METAL,"POLYGON Z ((665070.715 1712492.135 27.175, 665..."


In [4]:
in_file = '../data/rasters/Drone/drone_colihaut_DOM.tif'
exp_config = '../configs/config_04.yaml'
c = config.create_config(exp_config)
exp_dir = f"../exp/{c['exp_name']}/"
classes = geoutils.classes_dict[c['attribute']]
print(f"Config: {c}")

model = pred_utils.load_model(c, exp_dir, n_classes=len(classes))
bldgs = pred_utils.generate_predictions(bldgs, model, c, in_file, exp_dir, classes=classes)   
bldgs.head(3)

Config: {'data_dir': './data/rasters/Tiles/ortho/', 'csv_dir': './data/csv/', 'attribute': 'roof_type', 'exp_name': 'exp-04', 'mode': 'RGB', 'batch_size': 32, 'n_workers': 4, 'n_epochs': 30, 'model': 'inceptionv3', 'pretrained': True, 'scheduler': 'ReduceLROnPlateau', 'optimizer': 'Adam', 'lr': 1e-05, 'img_size': 299, 'momentum': None, 'gamma': None, 'step_size': None, 'patience': 7, 'dropout': None}
Model file ../exp/exp-04/best_model.pth successfully loaded.


100%|███████████████| 373/373 [06:07<00:00,  1.01it/s]                                                                 


,UID,roof_type,roof_material,geometry
4,34876,NO_ROOF,CONCRETE_CEMENT,"POLYGON Z ((665060.888 1712479.780 35.786, 665..."
5,34887,GABLE,HEALTHY_METAL,"POLYGON Z ((665078.259 1712486.585 32.366, 665..."
6,34890,GABLE,HEALTHY_METAL,"POLYGON Z ((665070.715 1712492.135 27.175, 665..."


In [13]:
in_file = '../data/rasters/DSM/ndsm_DOM.tif'
exp_config = '../configs/config_05.yaml'
c = config.create_config(exp_config)
exp_dir = f"../exp/{c['exp_name']}/"
classes = geoutils.classes_dict[c['attribute']]
print(f"Config: {c}")

model = pred_utils.load_model(c, exp_dir, n_classes=len(classes))
bldgs = pred_utils.generate_predictions(bldgs, model, c, in_file, exp_dir, classes=classes)   
bldgs.head(3)

Config: {'data_dir': './data/rasters/Tiles/ndsm/', 'csv_dir': './data/csv/', 'attribute': 'roof_type', 'mode': 'GRAYSCALE', 'exp_name': 'exp-05', 'batch_size': 32, 'n_workers': 4, 'n_epochs': 30, 'model': 'resnet50', 'pretrained': True, 'scheduler': 'ReduceLROnPlateau', 'optimizer': 'Adam', 'lr': 1e-05, 'img_size': 224, 'momentum': None, 'gamma': None, 'step_size': None, 'patience': 7, 'dropout': 0.5}
Model file ../exp/exp-05/best_model.pth successfully loaded.


100%|███████████████| 1272/1272 [11:39<00:00,  1.82it/s]                                                               


,UID,Elevation,Layer,roof_type,roof_material,roof_condition,shape_name,geometry
34411,34413,17.535746,BLD_GEN_OUTLINE,HIP,CONCRETE_CEMENT,NaN,Saint Peter,MULTIPOLYGON Z (((665583.468 1710787.345 14.89...
34476,34478,5.934477,BLD_GEN_OUTLINE,GABLE,CONCRETE_CEMENT,NaN,Saint Peter,MULTIPOLYGON Z (((665565.273 1710859.963 5.669...
34487,34489,6.699960,BLD_RESIDENTIAL_TRAILER,FLAT,CONCRETE_CEMENT,NaN,Saint Peter,MULTIPOLYGON Z (((665534.946 1710892.122 6.222...


In [6]:
name = 'Colihaut'
out_path = '../output/'
if not os.path.isdir(out_path):
    os.makedirs(out_path)
out_file = os.path.join(out_path, f'{name}-drone-20230616.gpkg')
bldgs.to_file(out_file, driver='GPKG')

## Model Evaluation

In [19]:
ytrue_file = '../data/vectors/building_footprints_annotated_COLIHAUT.gpkg'
ytrue = gpd.read_file(ytrue_file)[['UID', 'roof_type', 'roof_material', 'geometry']].dropna()
ytrue.roof_type = ytrue.roof_type.replace({'PYRAMID': 'HIP', 'HALF_HIP': 'HIP'})
print(f"Data dimensions: {ytrue.shape}")
print(ytrue.roof_type.value_counts())
print(ytrue.roof_material.value_counts())
ytrue.head(3)

Data dimensions: (373, 4)
roof_type
GABLE      247
FLAT        72
HIP         44
NO_ROOF     10
Name: count, dtype: int64
roof_material
HEALTHY_METAL      187
IRREGULAR_METAL    130
CONCRETE_CEMENT     46
INCOMPLETE          10
Name: count, dtype: int64


,UID,roof_type,roof_material,geometry
4,34876,NO_ROOF,INCOMPLETE,"POLYGON Z ((665060.888 1712479.780 35.786, 665..."
5,34887,GABLE,HEALTHY_METAL,"POLYGON Z ((665078.259 1712486.585 32.366, 665..."
6,34890,GABLE,IRREGULAR_METAL,"POLYGON Z ((665070.715 1712492.135 27.175, 665..."


In [18]:
ypred_file = f'../output/{name}-drone-20230616.gpkg'
ypred = gpd.read_file(ypred_file)[['UID', 'roof_type', 'roof_material', 'geometry']].dropna()
print(f"Data dimensions: {ypred.shape}")
print(ypred.roof_type.value_counts())
print(ypred.roof_material.value_counts())
ypred.head(3)

Data dimensions: (373, 4)
roof_type
GABLE      250
HIP         59
FLAT        54
NO_ROOF     10
Name: count, dtype: int64
roof_material
HEALTHY_METAL      204
IRREGULAR_METAL    113
CONCRETE_CEMENT     48
INCOMPLETE           7
BLUE_TARP            1
Name: count, dtype: int64


,UID,roof_type,roof_material,geometry
0,34876,NO_ROOF,CONCRETE_CEMENT,"POLYGON Z ((665060.888 1712479.780 35.786, 665..."
1,34887,GABLE,HEALTHY_METAL,"POLYGON Z ((665078.259 1712486.585 32.366, 665..."
2,34890,GABLE,HEALTHY_METAL,"POLYGON Z ((665070.715 1712492.135 27.175, 665..."


In [22]:
classes

['FLAT', 'GABLE', 'HIP', 'NO_ROOF']

In [26]:
attribute = 'roof_type'
classes = geoutils.classes_dict[attribute]
#cm = eval_utils.get_confusion_matrix(ytrue[attribute], ypred[attribute], classes)
eval_utils.evaluate(ytrue[attribute], ypred[attribute])

{'overall_accuracy': 89.27613941018767,
 'balanced_accuracy': 87.45253649858913,
 'f1_score_micro': 89.27613941018767,
 'f1_score': 86.36588137237665,
 'precision_score': 87.00772128060264,
 'recall_score': 87.45253649858913,
 'f1_per_class': array([80.95238095, 92.95774648, 81.55339806, 90.        ]),
 'precision_per_class': array([94.44444444, 92.4       , 71.18644068, 90.        ]),
 'recall_per_class': array([70.83333333, 93.52226721, 95.45454545, 90.        ])}

## Data Fusion

In [8]:
c = config.create_config("../configs/fusion_06.yaml")
c = {key: '.' + value if 'config' in key or 'dir' in key else value for key, value in c.items()}

c1 = config.create_config(c['config1'])
c1 = {key: '.' + value if 'dir' in key else value for key, value in c1.items()}
classes = geoutils.classes_dict[c1['attribute']]
exp_dir = os.path.join(c['exp_dir'], c1['exp_name'])
model1 = pred_utils.load_model(c1, exp_dir=exp_dir , n_classes=len(classes))
print(c1)

c2 = config.create_config(c['config2'])
exp_dir = os.path.join(c['exp_dir'], c2['exp_name'])
c2 = {key: '.' + value if 'dir' in key else value for key, value in c2.items()}
model2 = pred_utils.load_model(c2, exp_dir=exp_dir, n_classes=len(classes))
print(c2)

Model file ../exp/exp-09\best_model.pth successfully loaded.
{'data_dir': '../data/rasters/Tiles/ortho/', 'csv_dir': '../data/csv/', 'attribute': 'roof_material', 'exp_name': 'exp-09', 'mode': 'RGB', 'batch_size': 32, 'n_workers': 4, 'n_epochs': 30, 'model': 'efficientnetb0', 'pretrained': True, 'scheduler': 'ReduceLROnPlateau', 'optimizer': 'Adam', 'lr': 1e-05, 'img_size': 224, 'momentum': None, 'gamma': None, 'step_size': None, 'patience': 7, 'dropout': None}
Model file ../exp/exp-06\best_model.pth successfully loaded.
{'data_dir': '../data/rasters/Tiles/ndsm/', 'csv_dir': '../data/csv/', 'attribute': 'roof_material', 'exp_name': 'exp-06', 'mode': 'GRAYSCALE', 'batch_size': 32, 'n_workers': 4, 'n_epochs': 30, 'model': 'inceptionv3', 'pretrained': True, 'scheduler': 'ReduceLROnPlateau', 'optimizer': 'Adam', 'lr': 1e-05, 'img_size': 299, 'momentum': None, 'gamma': None, 'step_size': None, 'patience': 7, 'dropout': None}


In [101]:
source1 = '../data/rasters/Ortho/ortho_DOM.tif'
source2 = '../data/rasters/DSM/ndsm_DOM.tif'
data = fusion.predict(bldgs, c1, c2, model1, model2, source1=source1, source2=source2)

100%|███████████████| 522/522 [11:29<00:00,  1.32s/it]                                                                 


In [9]:
#features = fusion.get_features(c, data)
model_file = os.path.join(c['exp_dir'], c['exp_name'], c['mode'], c['model'], 'best_model.pkl')
model = joblib.load(model_file)
#preds = model.predict(data[features])
model

Pipeline(steps=[('scaler', MinMaxScaler()), ('selector', None),
                ('model',
                 RandomForestClassifier(bootstrap=False, criterion='entropy',
                                        max_depth=9, n_estimators=450,
                                        n_jobs=-1, random_state=42,
                                        verbose=1))])

In [7]:
?model.

Type:        Pipeline
String form:
Pipeline(steps=[('scaler', MinMaxScaler()), ('selector', None),
           ('model',
           <...> gression(C=0.1, max_iter=1000, n_jobs=-1,
           random_state=42))])
Length:      3
File:        c:\users\issa\miniconda3\envs\gfdrr\lib\site-packages\sklearn\pipeline.py
Docstring:  
Pipeline of transforms with a final estimator.

Sequentially apply a list of transforms and a final estimator.
Intermediate steps of the pipeline must be 'transforms', that is, they
must implement `fit` and `transform` methods.
The final estimator only needs to implement `fit`.
The transformers in the pipeline can be cached using ``memory`` argument.

The purpose of the pipeline is to assemble several steps that can be
cross-validated together while setting different parameters. For this, it
enables setting parameters of the various steps using their names and the
parameter name separated by a `'__'`, as in the example below. A step's
estimator may be replaced entire

In [104]:
bldgs['roof_material'] = preds
out_path = '../output/'
if not os.path.isdir(out_path):
    os.makedirs(out_path)
out_file = os.path.join(out_path, f'{name}-ortho-20230613.gpkg')
bldgs.to_file(out_file, driver='GPKG')

In [64]:
preds

array(['HEALTHY_METAL'], dtype=object)

In [ ]:
bldgs[bldgs.UID == 35171]